In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import math

import numpy as np
from scipy.spatial.transform import Rotation

from rozumarm_vima_utils.transform import (
    map_tf_repr,
    map_gripper_rf
)

In [ ]:
from rozumarm_vima_utils.robot import RozumArm, HOME_TCP_ANGLES, Z_PREP_LVL

In [ ]:
robot = RozumArm()

In [ ]:
# robot.api.open_gripper()
# robot.api.close_gripper()

### visit key points

In [ ]:
from rozumarm_vima_utils.transform import rf_tf_c2r


TABLE_FRAME_POINTS_FILEPATH = "assets/aruco_corners_top.npy"
ROZUM_FRAME_POINTS_FILEPATH = "assets/marker_points_in_rozum_rf_v1.npy"


key_points_crf = np.load(TABLE_FRAME_POINTS_FILEPATH)
key_points_crf = key_points_crf[:, :2]
key_points_rrf = np.load('assets/marker_points_in_rozum_rf_v2_synth.npy')

transformed_cam_points = np.stack([rf_tf_c2r(v) for v in key_points_crf])

In [ ]:
Z_BOT_BIAS = -0.085

i = 0
target_point = np.concatenate((key_points_rrf[i], [0.18 + 0.1 + Z_BOT_BIAS]))
robot._move_tcp(target_point, HOME_TCP_ANGLES)

In [ ]:
i = 3
target_point = np.concatenate((transformed_cam_points[i], [0.18 + 0.1 + Z_BOT_BIAS]))
robot._move_tcp(target_point, HOME_TCP_ANGLES)

### eef guiding

In [ ]:
pos = robot.api.get_position()
point = [getattr(pos.point, axis_name) for axis_name in ('x', 'y', 'z')]
rot = [getattr(pos.rotation, angle_name) for angle_name in ('roll', 'pitch', 'yaw')]

rot = [-math.pi, 0, 0]

# point[0] -= 0.0005
# point[1] += 0.005
# point[2] -= 0.01

robot._move_tcp(point, rot)

### test eef orientation during swipe

In [ ]:
vima_quat = robot.get_swipe_quat([0.0, 0.0], [0.1, 0.1])
vima_angles = Rotation.from_quat(vima_quat).as_euler('XYZ')
print(f'Desired rotation in VIMA-rf: {vima_angles}')

rozum_quat = map_tf_repr(map_gripper_rf(vima_quat))
rozum_angles = Rotation.from_quat(rozum_quat).as_euler('XYZ')
print(f'Desired rotation in Rozum-rf: {rozum_angles}')